In [7]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [50]:
import pandas as pd
import requests
from pymongo import MongoClient
from datetime import datetime


In [51]:
# URL des données
url_1 = 'https://airqino-api.magentalab.it/v3/getStationHourlyAvg/283181971'  # Remplacez par le lien réel
url_2 = 'https://airqino-api.magentalab.it/v3/getStationHourlyAvg/283164601'  # Remplacez par le lien réel


In [52]:
# Récupérer les données depuis le lien
response_1 = requests.get(url_1)
data_1 = response_1.json()
response_2 = requests.get(url_2)
data_2 = response_2.json()

In [53]:
# Convertir la liste de données en DataFrame
df_1 = pd.DataFrame(data_1["data"])
df_2 = pd.DataFrame(data_2["data"])

In [54]:
df_1['timestamp'] = pd.to_datetime(df_1['timestamp'])
df_2['timestamp'] = pd.to_datetime(df_2['timestamp'])

In [55]:
df_1['date_1'] = df_1['timestamp'].dt.date
df_2['date_2'] = df_2['timestamp'].dt.date

In [56]:
daily_means_1 = df_1.groupby('date_1').agg({'CO': 'mean', 'PM2.5': 'mean'}).reset_index()
daily_means_2 = df_2.groupby('date_2').agg({'CO': 'mean', 'PM2.5': 'mean'}).reset_index()

In [57]:
daily_means_1['date_1'] = pd.to_datetime(daily_means_1['date_1'])
daily_means_2['date_2'] = pd.to_datetime(daily_means_2['date_2'])


In [58]:
# Afficher les résultats
print(daily_means_1)
print(daily_means_2)

        date_1        CO      PM2.5
0   2023-09-10  0.056827  12.906667
1   2023-09-11  0.034917  11.078584
2   2023-09-12  0.227283  12.922088
3   2023-09-13  0.209933  14.277363
4   2023-09-14  0.150275  14.533692
..         ...       ...        ...
343 2024-09-05  0.139701  13.180421
344 2024-09-06  0.118769  14.648611
345 2024-09-07  0.107480  14.217473
346 2024-09-08  0.128420  12.615905
347 2024-09-09  0.199441  15.024671

[348 rows x 3 columns]
        date_2        CO     PM2.5
0   2023-09-10  0.000000  4.513333
1   2023-09-11  0.000000  3.541353
2   2023-09-12  0.000000  3.890143
3   2023-09-13  0.002679  5.342250
4   2023-09-14  0.000000  5.709946
..         ...       ...       ...
355 2024-09-05  0.000000  6.137948
356 2024-09-06  0.000000  7.079704
357 2024-09-07  0.000000  7.309319
358 2024-09-08  0.000000  6.057841
359 2024-09-09  0.000000  6.501135

[360 rows x 3 columns]


In [59]:
# Connexion à MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Remplacez par l'URI de votre MongoDB
db_1 = client['air_quality_db']
collection_1 = db_1['daily_averages_stationId_283181971']
db_2 = client['air_quality_db']
collection_2 = db_2['daily_averages_stationId_283164601']

In [60]:
# Convertir le DataFrame en dictionnaire et insérer les données dans MongoDB
records_1 = daily_means_1.to_dict('records')
collection_1.insert_many(records_1)

records_2 = daily_means_2.to_dict('records')
collection_2.insert_many(records_2)
print("Données insérées sans doublons.")

Données insérées sans doublons.
